<h1>Store 3266 Presub metrics analysis</h1>
<ul>starting week 10</ul>

In [1]:
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install torch torchvision torchaudio
!pip install scikit-learn
!pip install xgboost
!pip install lightgbm
!pip install tensorflow-macos
!pip install tensorflow-metal
!pip install keras-tuner
!pip install psutil

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chardet
import seaborn as sns
from datetime import datetime, timedelta, date 
import glob
import os
import time
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import xgboost as xgb
import platform
import re

notebook_start_time = time.time()

In [62]:
xlsx_dir = 'Data/DigitalSBU/'
xlsx_files = glob.glob(os.path.join(xlsx_dir, "DIGITALSBU_*.xlsx"))
xlsx_files = sorted(xlsx_files)

dataframes = {}

# Extract the week number from the last file in the sorted list
match = re.search(r'WEEK(\d+)', xlsx_files[-1])
if match:
    week_number = match.group(1)
    print("Latest week number:", week_number)

# Load each Excel file into the dictionary
for file_path in xlsx_files:
    try:
        df = pd.read_excel(file_path)
        filename = os.path.basename(file_path).replace('.xlsx', '_df')
        dataframes[filename] = df  # ✅ Store DataFrame with filename as key
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# ✅ Print the keys of the dictionary
print("DataFrames loaded:")
for name in dataframes:
    print(name)

Latest week number: 13
DataFrames loaded:
DIGITALSBU_3266_WEEK10_df
DIGITALSBU_3266_WEEK11_df
DIGITALSBU_3266_WEEK12_df
DIGITALSBU_3266_WEEK13_df


In [65]:
most_recent_week = max(dataframes.keys(), key=lambda x: int(re.search(r'WEEK(\d+)', x).group(1)))
print(most_recent_week)

DIGITALSBU_3266_WEEK13_df


In [67]:
dataframes[most_recent_week]

,SBU,Dept.,Dept Description,First Time Pick % TY,Pre-Sub % TY,Post Sub % TY,Items Ordered TY,Items Picked Qty TY,Items Picked % TY,Exceptions Qty TY,Exceptions % TY,Substituted Qty TY,Substituted % TY,Nil Qty TY,Nil Pick % TY
0,Consumables,NaN,NaN,0.919261,0.939633,0.981714,1883,1801,0.956452,11.0,0.005842,35,0.018587,95,0.050451
1,NaN,2.0,Health & Beauty Aids,0.925900,0.965400,0.994900,428,427,0.997700,7.0,0.016400,10,0.023400,9,0.021000
2,NaN,4.0,Household Paper,0.971400,0.989300,0.994400,280,277,0.989300,NaN,NaN,0,0.000000,3,0.010700
3,NaN,8.0,Pets & Supplies,0.955800,0.964900,1.000000,114,107,0.938600,NaN,NaN,0,0.000000,4,0.035100
4,NaN,13.0,Household Chemicals,0.971500,0.978900,0.993700,283,281,0.992900,NaN,NaN,5,0.017700,4,0.014100
5,NaN,40.0,OTC Pharmacy,0.855000,0.882500,0.978400,396,357,0.901500,4.0,0.010100,17,0.042900,38,0.096000
6,NaN,46.0,Cosmetics & Skincare,0.876600,0.882800,0.884100,230,210,0.913000,NaN,NaN,2,0.008700,26,0.113000
7,NaN,79.0,Infant Consumables,0.915600,0.922100,0.972600,152,142,0.934200,NaN,NaN,1,0.006600,11,0.072400
8,Food,NaN,NaN,0.932099,0.948174,0.971640,4062,3944,0.970950,23.0,0.005662,83,0.020433,100,0.024618
9,NaN,1.0,Candy & Tobacco,0.918500,0.933300,0.936500,127,124,0.976400,NaN,NaN,0,0.000000,8,0.063000
